### Setting the Stage

Consider the scenario:

Let's say you have an Oracle database and a MySQL database each with multiple schemas and each one of these schemas have multiple tables. You are working on a project that needs different tables from these different databases. You need to carry your own investigation and check for table metadata; column names, data type and range. There is software that will allow you to conduct this investigation but hardly enough automation or easy ways to take notes on discoveries.

You could do this investigation with SQLAlchemy, in a jupyter notebook if you want and easily take notes all without switching from tool to tool.

Say furthermore that you worked on a project in MySQL and used a MySQL only and dedicated python package. You had multiline SQL queries and hardcoded different filters. One day your company decides to migrate to something different; say [snowflake](https://www.snowflake.com/product/architecture/) for example. Your models need to be tuned and your data processing will need to be adjusted. You have to re-create some of the tables you were in charge of, but obviously some of the queries you wrote will need to be changed! You will probably have a hard time refactoring your code, sql queries etc. But with SQLAlchemy, it will be easier. Your tables can be re-created with nearly no additional effort. You can update/tune your filters (as we will see later) and all will go as planned. 

From my experience, using SQLAlchemy has helped with metadata investigation, consistency with different SQL dialects and it has made refactoring and maintenance extremely easier than expected. These are reasons why I personally love using SQLAlchemy whenever possible.

### Why SQLAlchemy?

SQLAlchemy and ORM packages are very popular with pythond developers working on non-data science related software/webapps. I think one of the main reasons are "protection against sql injection attacks" but it is database agnostic and as such, it allows for great consistency and light maintenance during database migrations. The following 4 points are my main reasons for using SQLAlchemy.

<ol>
    <li>Easy access to table metadata</li>
    <li>Consitent cursor/engine methods</li>
    <li>Better control over table architecture</li>
    <li>Refractoring is made EASY</li>
</ol>

Notes:
<ol>
    <li>
        There are methods and objects that come with SQLAlchemy that allow the user to quickly derive table metadata. This saves time from switching between your python IDE and Oracle SQL Developer for example.
    </li>
    <li>
        Interfacing with databases is not always straigh forward in python. Although the [DBAPI](https://www.python.org/dev/peps/pep-0249/) specs provides a standard, different developers are free to structure/name their methods as they see fit.
    </li>
    <li>
        There is nothing like having great control over the structure of the table you are responsible for. You must make sure that all primary keys are properly identified, that all columns that cannot be null are distinguishable and that database defaults are set.
    </li>
    <li>
        If you use an IDE like Pycharm, you probably love how easy it is to refractor objects. Refractoring objects/variables are very easy and the fact that SQLAlchemy provides the object representation of tables and columns; refractoring is often (not always) a piece of cake.
    </li>
    
</ol>

### Con
The main complaint is always efficiency/speed of sqlalchemy compared to raw sql paired with dedicated packages for each SQL dialect.

### Setup

In [1]:
import pathlib
import pandas as pd
import sqlalchemy as sa
import sqlalchemy.orm as orm
from sqlalchemy.engine import reflection
from sqlalchemy.ext.hybrid import hybrid_property
from sqlalchemy.ext.declarative import declarative_base

In [2]:
# create a database
db_file = pathlib.Path.cwd() / "temp_db.db"
db_file.touch()
print(db_file)

/Users/josiahhounyo/Documents/GitHub/uchicago/temp_db.db


In [3]:
engine = sa.create_engine(f"sqlite:///{db_file}")
ny_sat = r"https://data.cityofnewyork.us/api/views/zt9s-n5aj/rows.csv?accessType=DOWNLOAD"
data_docs = r"https://catalog.data.gov/dataset/sat-college-board-2010-school-level-results-5c6d6"

Please go to https://sqlitebrowser.org/dl/ and install the sqlite browser

After installation, click on **Open Database** and open the file created

In [4]:
df = pd.read_csv(ny_sat)
df.columns = [column.lower().replace(' ', '_') for column in df.columns]
df.head()

,dbn,school_name,number_of_test_takers,critical_reading_mean,mathematics_mean,writing_mean
0,01M292,Henry Street School for International Studies,31.0,391.0,425.0,385.0
1,01M448,University Neighborhood High School,60.0,394.0,419.0,387.0
2,01M450,East Side Community High School,69.0,418.0,431.0,402.0
3,01M458,SATELLITE ACADEMY FORSYTH ST,26.0,385.0,370.0,378.0
4,01M509,CMSP HIGH SCHOOL,NaN,NaN,NaN,NaN


### Metadata - Inspecting Data Available in a Database

In [5]:
inspector = reflection.Inspector.from_engine(engine)

# If we want all the schemas we have access to we can do this
inspector.get_schema_names()

['main']

In [6]:
# To get all the tables in a schema we can do the following
inspector.get_table_names(schema='main')

[]

In [7]:
# Let's add the data downloaded
df.to_sql('pandas_table', engine, index=False, if_exists='replace') 
# you can check the data in your sqlite database browser (we could have used indices as key)

In [8]:
# Reflecting tables
metadata = sa.MetaData(engine)
pandas_table = sa.Table('pandas_table', metadata, autoload_with=engine)
pandas_table

Table('pandas_table', MetaData(bind=Engine(sqlite:////Users/josiahhounyo/Documents/GitHub/uchicago/temp_db.db)), Column('dbn', TEXT(), table=<pandas_table>), Column('school_name', TEXT(), table=<pandas_table>), Column('number_of_test_takers', FLOAT(), table=<pandas_table>), Column('critical_reading_mean', FLOAT(), table=<pandas_table>), Column('mathematics_mean', FLOAT(), table=<pandas_table>), Column('writing_mean', FLOAT(), table=<pandas_table>), schema=None)

In [9]:
list(pandas_table.columns)

[Column('dbn', TEXT(), table=<pandas_table>),
 Column('school_name', TEXT(), table=<pandas_table>),
 Column('number_of_test_takers', FLOAT(), table=<pandas_table>),
 Column('critical_reading_mean', FLOAT(), table=<pandas_table>),
 Column('mathematics_mean', FLOAT(), table=<pandas_table>),
 Column('writing_mean', FLOAT(), table=<pandas_table>)]

In [10]:
# column names - let's inspect a column object
column = pandas_table.columns['dbn'] # you can access columns using .columns or .c
type(column)

sqlalchemy.sql.schema.Column

In [11]:
print(
    f"""column name: {column.name}, type: {column.type}, primary key: {column.primary_key}, python type: {column.type.python_type}"""
)

column name: dbn, type: TEXT, primary key: False, python type: <class 'str'>


### Consistent Cursor/Engine Methods

Inserting or Updating data in a table is usually done via SQL queries. The specific syntax could differ from one dialect to another, Columns with special characters (spaces etc...) will need to be handled properly and on top of that, parsing data from a pandas dataframe to complete a sql query is not always straighforward.

Consider the following dataframe. How would you insert the data into the table?

In [12]:
data_to_insert_into_table = pd.DataFrame({
    'dbn': ['01M998', '01M999'],
    'school_name': ['My Made Up School', 'New School In Town']
})

data_to_update_in_table = pd.DataFrame({
    'dbn_to_update': ['01M292', '01M448'],
    'school_name_to_update': ['Updated School #1', 'Updated School #2']
})

In [13]:
data_to_update_in_table

,dbn_to_update,school_name_to_update
0,01M292,Updated School #1
1,01M448,Updated School #2


In [14]:
# You could do something like this
sql = """
insert into pandas_table (dbn, school_name) values (:1, :2)
"""

results = engine.execute(sql, ('01M997', 'School Just Newly Added'))
# check the data in your browser - you can refresh it and check the end

In [15]:
type(results)

sqlalchemy.engine.result.ResultProxy

In [16]:
# you can tell that you inserted one record
results.rowcount

1

In [17]:
# closer look at native properties of SQLAlchemy Table objects - very similar to what we had
print(pandas_table.insert())

INSERT INTO pandas_table (dbn, school_name, number_of_test_takers, critical_reading_mean, mathematics_mean, writing_mean) VALUES (?, ?, ?, ?, ?, ?)


In [18]:
# update properties - not very sufficient here!
print(pandas_table.update())

UPDATE pandas_table SET dbn=?, school_name=?, number_of_test_takers=?, critical_reading_mean=?, mathematics_mean=?, writing_mean=?


Dictionaries are key to inserting and updating using these methods and Pandas have great support for [turning the data into dictionaries](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_dict.html#pandas.DataFrame.to_dict).

In [19]:
data_to_insert_into_table.to_dict(orient='records') # I believe record as reference to database/table record?

[{'dbn': '01M998', 'school_name': 'My Made Up School'},
 {'dbn': '01M999', 'school_name': 'New School In Town'}]

In [20]:
_ = engine.execute(pandas_table.insert(), data_to_insert_into_table.to_dict(orient='records'))
# check the database in your explorer - at the end of the data

In [21]:
# for updates, you need to specify which fields you want to check and those you want to set
update_statement = pandas_table.update().where(
    pandas_table.columns['dbn'] == sa.bindparam('dbn_to_update')
).values(school_name=sa.bindparam('school_name_to_update'))

print(update_statement)

UPDATE pandas_table SET school_name=? WHERE pandas_table.dbn = ?


In [22]:
data_to_update_in_table.to_dict(orient='records')

[{'dbn_to_update': '01M292', 'school_name_to_update': 'Updated School #1'},
 {'dbn_to_update': '01M448', 'school_name_to_update': 'Updated School #2'}]

In [23]:
# What is bindparam doing?
# What if I have a more complex where clause?
# What if I had more values to set?

In [24]:
_ = engine.execute(update_statement, data_to_update_in_table.to_dict(orient='records'))
# check the first two rows

Benefits of the approach above

<ul>
    <li>Database Agnostic</li>
    <li>Auto-generated table objects and insert/update statement with all escaping taken care of!</li>
    <li>Quickly and Easily turn Pandas Dataframes into Actual Records on Database</li>
    ...
</ul>

Keep in mind that there are some columns of certain types, datetime for example, can cause problems but that usually just requires further data processing.

### Table Structure

When you have to create/maintain tables, you MUST make sure constraints are not violated. If you inspect the data structure in your sqlite browser, you will notice that there are no primary keys. PRIMARY KEYS are sometimes necessary for the integrity of the data. Let's now create the table with two columns as primary keys.

In [25]:
# drop the previous table
pandas_table.drop(engine, checkfirst=True)

In [26]:
# Answer here
sql = """
CREATE TABLE IF NOT EXISTS ny_sat(
   dbn VARCHAR(15),
   school_name VARCHAR(100),
   number_of_test_takers INT,
   critical_reading_mean DOUBLE,
   mathematics_mean DOUBLE,
   writing_mean DOUBLE,
   PRIMARY KEY (dbn, school_name)
)
"""

engine.execute(sql)

In [27]:
engine.table_names()

['ny_sat']

With more columns and more constraints, the query can get longer and harder to read. I've seen some rather long table specifications in sql and they can be hard to maintain. But a fun fact is that some dialects allow/require different syntax on how to create tables. So if you were to switch dialects, you could face minor headaches. SQLAlchemy can help!

In [28]:
Base = declarative_base()

class NewYorkCitySat(Base):
    __tablename__ = 'new_york_sat'
    __table_args__ = {'schema' : None} # this is if you want to specify the schema
    
    dbn = sa.Column(sa.String(15), primary_key=True)
    school_name = sa.Column(sa.String(100), primary_key=True)
    number_of_test_takers = sa.Column(sa.Integer, nullable=True)
    critical_reading_mean = sa.Column(sa.Float, nullable=True)
    mathematics_mean = sa.Column(sa.Float, nullable=True)
    writing_mean = sa.Column(sa.Float, nullable=True)
    
    # since this is a class, we could provide an __init__ just because
    def __init__(self, dbn, school_name, number_of_test_takers, critical_reading_mean, mathematics_mean,
                writing_mean):
        self.dbn = dbn
        self.school_name = school_name
        self.number_of_test_takers = number_of_test_takers
        self.critical_reading_mean = critical_reading_mean
        self.mathematics_mean = mathematics_mean
        self.writing_mean = writing_mean
    
    # say we want to have some sort of computation that tells us when a school is a "good" school
    # the criteria for this is all made up and could be complicated but we focus on an easy one
    @hybrid_property
    def good_school(self):
        return self.critical_reading_mean >= 466 and self.mathematics_mean >= 489 and self.writing_mean >= 464
    
    # We obviously could not use that propery directy as a filter on a query on the database
    @good_school.expression
    def good_school(cls):
        return sa.and_(
            cls.critical_reading_mean >= 466,
            cls.mathematics_mean >= 489,
            cls.writing_mean >= 464
        )
        
# there are others types for the columns. There are also ways to set server default values!
# you could add other methods to the class. You could even add custom validators (only for instances of the class)

We could actually create a `Table` object but instead we used the ORM approach. Just a heads up, `np.nan` does not play well with these methods (although I believe you could configure sqlalchemy to treat them as `None`) so we will be replacing them with `None` since `None` is the real equivalent of `NULL`.

In [29]:
# That might have been tedious yes. But it not only has an easy to understand object, it also has the easy to use
# Table object accessible like this

NewYorkCitySat.__table__

Table('new_york_sat', MetaData(bind=None), Column('dbn', String(length=15), table=<new_york_sat>, primary_key=True, nullable=False), Column('school_name', String(length=100), table=<new_york_sat>, primary_key=True, nullable=False), Column('number_of_test_takers', Integer(), table=<new_york_sat>), Column('critical_reading_mean', Float(), table=<new_york_sat>), Column('mathematics_mean', Float(), table=<new_york_sat>), Column('writing_mean', Float(), table=<new_york_sat>), schema=None)

In [30]:
# one way of creating the table on the database
NewYorkCitySat.__table__.create(bind = engine, checkfirst = True)

# equivalent to
Base.metadata.tables['new_york_sat'].create(bind = engine, checkfirst = True)

# another way
Base.metadata.create_all(bind = engine, tables = [NewYorkCitySat.__table__], checkfirst = True)

# preference is left to the user - oh if the second command didn't have checkfirst, there would be an error
# try it and see the error message, take a peek at the sql query used

In [31]:
# Creating instances of the class -  this is what we would do without the __init__ method
one_record = NewYorkCitySat(
    dbn = '01M292',
    school_name = 'My High School',
    number_of_test_takers = 12,
    critical_reading_mean = 190,
    mathematics_mean = 200,
    writing_mean = 180
)

type(one_record)

__main__.NewYorkCitySat

In [32]:
# Creating instances by relying on the __init__ method since we have it
other_record = NewYorkCitySat('01M292', 'My High School', 12, 190, 200, 180)

the only difference is that without the \_\_init\_\_ method, we need to create records as keyword arguments but with the \_\_init\_\_ method, we can use positional arguments. We could even do some final processing by record (though it's always best to do the processing at DataFrame level)

In [33]:
# Inserting the data using orm sessions - create instances from the dataframe
session = orm.Session(engine)
session.add(one_record)
session.commit()
session.close()

# check the new_york_sat table in the database browser

In [34]:
# what happens when we run this?
try:
    session = orm.Session(engine)
    session.add(other_record)
    session.commit()
    session.close()
except Exception as e:
    print(e.args[0])

(sqlite3.IntegrityError) UNIQUE constraint failed: new_york_sat.dbn, new_york_sat.school_name


We get the error because we violated the primary key constraint. This can be a lifesaver because had we not added primary keys, we would silently introduce duplicate rows.

In [35]:
# adding many records
session.add_all([one_record, other_record])

# sessions can be rolled back so they provide some safety
session.rollback()

/Users/josiahhounyo/anaconda3/envs/dsdbtest/lib/python3.7/site-packages/sqlalchemy/orm/session.py:552: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  "Session's state has been changed on "


Adding the whole dataframe to the database. We already know that we can turn our dataframe into a list of records. We also must take of care `np.nan`s

In [36]:
df.to_dict(orient='records')[0]

{'dbn': '01M292',
 'school_name': 'Henry Street School for International Studies ',
 'number_of_test_takers': 31.0,
 'critical_reading_mean': 391.0,
 'mathematics_mean': 425.0,
 'writing_mean': 385.0}

In [37]:
# creating records out of all the rows in the dataframe
records_to_add = [NewYorkCitySat(**record) for record in df.to_dict(orient='records')]
records_to_add[0]

In [38]:
# remember that we had our __init__ method. we could do this mainly because we defined our columns in the same
# order as the dataframe :)
records_to_add_using_init = [NewYorkCitySat(*values) for values in df.values]
records_to_add_using_init[0]

In [39]:
records_to_add = [NewYorkCitySat(**record) for record in df.where(pd.notnull(df), None).to_dict(orient='records')]
session = orm.Session(engine)
session.add_all(records_to_add)
session.commit()
session.close()

### Refactoring and Easy Maitenance

My personal experience is that refactoring sql queries can be tedious. When you build a complex query that spans multiple lines, changing even one column in that string can cause major problems (or at least make debugging a nightmare). Building queries with the metadata classes on the other hand makes it easy to make changes to the query.

Say I want to select all schools that are good. **Good means that critical reading mean is at least 466, mathematics mean is at least 489 and writing mean is at least 464**.

In [40]:
sql = """
select * from new_york_sat where critical_reading_mean >= 466 and mathematics_mean >= 489
and writing_mean >= 464
"""

pd.read_sql(sql, engine)

,dbn,school_name,number_of_test_takers,critical_reading_mean,mathematics_mean,writing_mean
0,01M539,"New Explorations into Sci, Tech and Math HS",47,568.0,583.0,568.0
1,01M696,BARD HIGH SCHOOL EARLY COLLEGE,138,630.0,608.0,630.0
2,02M411,Baruch College Campus High School,96,523.0,583.0,528.0
3,02M412,New York City Laboratory School Collab Studies,108,561.0,597.0,567.0
4,02M416,ELEANOR ROOSEVELT HIGH SCHOOL,122,555.0,596.0,567.0
5,02M418,Millennium High School,140,512.0,554.0,523.0
6,02M475,STUYVESANT HIGH SCHOOL,804,674.0,735.0,678.0
7,03M479,BEACON SCHOOL,237,573.0,563.0,575.0
8,03M485,LAGUARDIA HIGH SCH MUSIC ART,594,558.0,555.0,567.0
9,03M541,MANHATTAN/HUNTER COLL HS SCI,80,481.0,525.0,469.0


In [41]:
# now let's look at the good_school expression we created. It is exactly what we would normally write
# each time we have to use the condition as a filter
print(NewYorkCitySat.good_school.expression)

new_york_sat.critical_reading_mean >= :critical_reading_mean_1 AND new_york_sat.mathematics_mean >= :mathematics_mean_1 AND new_york_sat.writing_mean >= :writing_mean_1


In [42]:
# Create a query using goog_school filter - as mentioned, the threshold could change and case statements added
# we just define the good_school condition at that one place and make use of it everywhere that needs it

session = orm.Session()
query = session.query(NewYorkCitySat).filter(NewYorkCitySat.good_school)
print(query.statement)

# If I had a models.py with the class defined, I could update my good_school property
# and all my code that depend on it will remain the same

SELECT new_york_sat.dbn, new_york_sat.school_name, new_york_sat.number_of_test_takers, new_york_sat.critical_reading_mean, new_york_sat.mathematics_mean, new_york_sat.writing_mean 
FROM new_york_sat 
WHERE new_york_sat.critical_reading_mean >= :critical_reading_mean_1 AND new_york_sat.mathematics_mean >= :mathematics_mean_1 AND new_york_sat.writing_mean >= :writing_mean_1


In [43]:
pd.read_sql(query.statement, engine).shape # equivalent to len(query.all()) with session bound to engine

(30, 6)

In [44]:
pd.read_sql(query.statement, engine).head()

,dbn,school_name,number_of_test_takers,critical_reading_mean,mathematics_mean,writing_mean
0,01M539,"New Explorations into Sci, Tech and Math HS",47,568.0,583.0,568.0
1,01M696,BARD HIGH SCHOOL EARLY COLLEGE,138,630.0,608.0,630.0
2,02M411,Baruch College Campus High School,96,523.0,583.0,528.0
3,02M412,New York City Laboratory School Collab Studies,108,561.0,597.0,567.0
4,02M416,ELEANOR ROOSEVELT HIGH SCHOOL,122,555.0,596.0,567.0


If I had more queries all based on that one filtering condition, once the condition changes and I have to involve things like case statements; I will have to remember where I used the condition and properly adjust it everywhere. I can easily make mistakes and have unbalanced parentheses etc.

But with the ORM defined with the filtering as a property, I can focus on changing the condition as I see fit and all queries that use it will work well

In [45]:
if db_file.exists():
    engine.dispose()
    db_file.unlink()